In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf,keras
import tensorflow_federated as tff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data=pd.read_csv('USA.csv')
data.head(5)

,Power_1,Power_2,Power_3,Power_4,Power_5,Power_6,Power_7,Power_8,Power_9,Power_10,...,GSNR_71,GSNR_72,GSNR_73,GSNR_74,GSNR_75,GSNR_76,No. Spans,Total Distance(m),GSNR_1,GSNR_class
0,0.000000,0.000000,0.000096,0.000096,0.000096,0.000000,0.000096,0.0,0.000096,0.0,...,0.000000,0.000000,0.000000,26.391513,0.000000,0.000000,23,2050305,0.000000,0
1,0.000000,0.000096,0.000000,0.000096,0.000000,0.000096,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,25.764675,0.000000,0.000000,0.000000,23,2050305,0.000000,0
2,0.000000,0.000000,0.000000,0.000096,0.000000,0.000000,0.000000,0.0,0.000096,0.0,...,25.759626,0.000000,0.000000,0.000000,24.616361,24.297057,23,2050305,0.000000,0
3,0.000096,0.000096,0.000096,0.000096,0.000096,0.000000,0.000000,0.0,0.000096,0.0,...,21.235013,22.810189,0.000000,0.000000,0.000000,0.000000,23,2050305,22.651367,1
4,0.000000,0.000000,0.000096,0.000000,0.000096,0.000000,0.000000,0.0,0.000096,0.0,...,22.666472,21.211149,21.131602,21.349516,22.572032,0.000000,23,2050305,0.000000,0


In [3]:
train_data,test_data=train_test_split(data,test_size=0.2,random_state=42)

In [4]:
train_data.shape

(26400, 307)

In [5]:
test_data.shape

(6600, 307)

In [6]:
train_data.head(5)

,Power_1,Power_2,Power_3,Power_4,Power_5,Power_6,Power_7,Power_8,Power_9,Power_10,...,GSNR_71,GSNR_72,GSNR_73,GSNR_74,GSNR_75,GSNR_76,No. Spans,Total Distance(m),GSNR_1,GSNR_class
7223,0.000000,0.000094,0.000094,0.000000,0.000094,0.000000,0.000000,0.000000,0.000094,0.000000,...,16.353010,17.166413,0.0,0.000000,0.000000,0.000000,25,2302221,0.000000,0
5196,0.000000,0.000095,0.000000,0.000095,0.000000,0.000094,0.000094,0.000000,0.000094,0.000094,...,0.000000,16.937779,0.0,0.000000,17.770900,0.000000,30,2665646,0.000000,0
6547,0.000000,0.000000,0.000095,0.000000,0.000095,0.000000,0.000095,0.000000,0.000000,0.000000,...,0.000000,17.108380,0.0,17.686473,0.000000,0.000000,25,2302221,0.000000,0
26890,0.000097,0.000097,0.000097,0.000097,0.000000,0.000000,0.000097,0.000097,0.000097,0.000097,...,38.102858,38.508915,0.0,0.000000,0.000000,0.000000,11,925829,36.683427,1
30152,0.000092,0.000000,0.000000,0.000000,0.000091,0.000091,0.000000,0.000000,0.000091,0.000091,...,10.363795,0.000000,0.0,0.000000,10.518718,10.630682,44,3980044,11.111929,0


In [7]:
train_features=train_data.iloc[:,:-1]
train_labels=train_data.iloc[:,-1]

In [8]:
train_labels.head(5)

7223     0
5196     0
6547     0
26890    1
30152    0
Name: GSNR_class, dtype: int64

In [9]:
test_features=test_data.iloc[:,:-1]
test_labels=test_data.iloc[:,-1]

In [10]:
test_labels.head(5)

7155     0
16953    0
31771    0
9585     0
9176     0
Name: GSNR_class, dtype: int64

In [11]:
scalar=StandardScaler()
train_scaled=scalar.fit_transform(train_features)
test_scaled=scalar.transform(test_features)

In [12]:
scaled_train_df=pd.DataFrame(train_scaled,columns=train_features.columns)

In [13]:
scaled_test_df=pd.DataFrame(test_scaled,columns=test_features.columns)

In [14]:
train_labels.head(5)

7223     0
5196     0
6547     0
26890    1
30152    0
Name: GSNR_class, dtype: int64

In [15]:
train_data_model=tf.data.Dataset.from_tensor_slices((scaled_train_df.values,train_labels.to_frame().values),name="Training_Data")

In [16]:
test_data_model=tf.data.Dataset.from_tensor_slices((scaled_test_df.values,test_labels.to_frame().values),name="Testing_Data")

In [17]:
train_data_model

<_TensorSliceDataset element_spec=(TensorSpec(shape=(306,), dtype=tf.float64, name=None), TensorSpec(shape=(1,), dtype=tf.int64, name=None))>

In [18]:
test_data_model

<_TensorSliceDataset element_spec=(TensorSpec(shape=(306,), dtype=tf.float64, name=None), TensorSpec(shape=(1,), dtype=tf.int64, name=None))>

In [19]:
def input_shapes():
    return (tf.TensorSpec([None,306],tf.float64),
            tf.TensorSpec([None,1],tf.int64)
            )

In [189]:
def model_fn():
    model=tf.keras.Sequential([
        tf.keras.layers.Dense(256,activation='relu',input_shape=(306,),kernel_initializer='he_normal'),
        tf.keras.layers.Dense(128,activation='relu',kernel_initializer='he_normal'),
        tf.keras.layers.Dense(64,activation='relu',kernel_initializer='he_normal'),
        tf.keras.layers.Dense(32,activation='relu',kernel_initializer='he_normal'),
        tf.keras.layers.Dense(16,activation='relu',kernel_initializer='he_normal'),
        tf.keras.layers.Dense(1,activation='sigmoid',kernel_initializer='he_normal')
    ],name='USA-Teacher')
    return tff.learning.from_keras_model(
        model,
        input_spec=input_shapes(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
        )
    

In [ ]:
trainer = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.Adam(),
    server_optimizer_fn=lambda: tf.keras.optimizers.Adam()
)

In [3]:
import sys
sys.version

'3.10.7 (tags/v3.10.7:6cc6b13, Sep  5 2022, 14:08:36) [MSC v.1933 64 bit (AMD64)]'